# The Dataframe Data Structure

The data scientist's primary tool is the dataframe.  This data structure enables the user to quickly filter and select records, as well as complete transformations, similar to SQL statements, but typically in a more expressible manner.  Once the data are in the correct format, it can be used as the input for a statistical or machine learning model.

The dataframe is described as rows of records, or observations, and columns of data variables describing those records.  It was originally implemented in the R language; however, it is now ubiquitous in data science frameworks, such as Python's Pandas, Scala's Spark, and Java's TableSaw.  This section reviews Pandas and compares it against the R dataframe when useful.

## Introduction

In the Datascience R versus Pandas debate, it is really an apples and oranges comparison.  R is a domain specific language in the field of statistics, analytics, and data visualization.  This makes R great for consulting, research, and basic analysis, especially within a careful academic context.  

In contrast, Python's statistics packages are woefully inadequate and rarely mention details which are of great importance to statistical practicioners.  An example of this is the use of contrasts in linear models.  The different Types (I-IV) of Analysis Of Variance models use different encodings for data.  Determining their estimators is not trivial.

However, if you want tight integration with other applications, the strengths of typical programming languages, and want to 'just get stuff done', then Python / Pandas is a great solution.  Pandas is quite good at data manipulation.  Python has the very strong NumPy and SciKit Learn module, which are very good for matrix operations and predictive modeling.  And the Python language is a really good general scripting language with strong support for strings and datetime types.

## Config

We will begin by installing both the jupyter `R-irkenel` and `rpy2` so that we can move data between R and Pandas and compare expressions and results.

In [2]:
import pandas as pd
import numpy as np

! pip install rpy2
! #the following may not be necessary depending on the version
! cp /usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.30 /opt/conda/lib/python3.9/site-packages/zmq/backend/cython/../../../../.././

In [3]:
%load_ext rpy2.ipython

In [4]:
trades = pd.DataFrame(
    [
        ["2016-05-25 13:30:01.023", "MSFT", 51.95, 75],
        ["2016-05-25 13:30:01.038", "MSFT", 51.95, 155],
        ["2016-05-25 13:30:03.048", "GOOG", 720.77, 100],
        ["2016-05-25 13:30:03.048", "GOOG", 720.92, 100],
        ["2016-05-25 13:30:03.048", "AAPL", 98.00, 100],
    ],
    columns=["timestamp", "ticker", "price", "quantity"],   #set index during assignment: `, index_col='timestamp'`
)
trades['timestamp'] = pd.to_datetime(trades['timestamp'])
trades.head()

,timestamp,ticker,price,quantity
0,2016-05-25 13:30:01.023,MSFT,51.95,75
1,2016-05-25 13:30:01.038,MSFT,51.95,155
2,2016-05-25 13:30:03.048,GOOG,720.77,100
3,2016-05-25 13:30:03.048,GOOG,720.92,100
4,2016-05-25 13:30:03.048,AAPL,98.00,100


We then set the index to the timestamp.  The index allows selecting rows with the `loc` and `at` methods.

In [5]:
trades_idx = trades.set_index('timestamp')
trades_idx.head()

,ticker,price,quantity
timestamp,,,
2016-05-25 13:30:01.023,MSFT,51.95,75
2016-05-25 13:30:01.038,MSFT,51.95,155
2016-05-25 13:30:03.048,GOOG,720.77,100
2016-05-25 13:30:03.048,GOOG,720.92,100
2016-05-25 13:30:03.048,AAPL,98.00,100


In [6]:
%%R -i trades
head( trades )

            timestamp ticker  price quantity
0 2016-05-25 13:30:01   MSFT  51.95       75
1 2016-05-25 13:30:01   MSFT  51.95      155
2 2016-05-25 13:30:03   GOOG 720.77      100
3 2016-05-25 13:30:03   GOOG 720.92      100
4 2016-05-25 13:30:03   AAPL  98.00      100


Everything looks to be working, let's move on.

## Numpy Arrays

The two primary classes we are interested in is the DataFrame and the Series, which are used as DataFrame columns.  Both the Series and DataFrame are numpy array with additional attributes and methods to integrate it with typical data science work.  We usually import numpy with pandas (as we did in above) so that we can flexibly change between the interfaces.  

Spending some time with the numpy module is very important for what is happening under the hood of pandas.  We will just show a few features, here, because they are extend to features of pandas.  Its important to keep in-mind the following:

* numpy is built on C++ for efficiency of memory and processing speed
* numpy arrays must be of homogenous data types
* because they are created with a fixed size, they must be recreated if the size changes
* arrays are founded on mathematical matrices and enable corresponding operations

The `ndarray` class is referred to as an “N-dimensional array”, so either a matrix or vector.  

After getting accustomed to the numpy functionality, you may find yourself reaching for it instead of a list.  This tendency should be tempered by a few aspects.  Importantly, numpy must be installed as it does not come built-in to Python.  Also, it is fairly heavy in size on disk.

### Creation


Let's see some basic numpy functionality, such as creating a sequence of numbers.

In [6]:
np.arange(4)

array([0, 1, 2, 3])

In [7]:
np.arange(2, 9, 2)

array([2, 4, 6, 8])

In [8]:
np.linspace(0, 10, num=5)

array([ 0. ,  2.5,  5. ,  7.5, 10. ])

Numpy describes matrices uses axes.  Many operations in numpy and pandas will be performed on axis 0 (rows) or 1 (cols).

In [9]:
#vector
arr = np.array([2, 1, 5, 3, 7, 4, 6, 8])
np.sort(arr)

array([1, 2, 3, 4, 5, 6, 7, 8])

In [10]:
arr.shape

(8,)

In [15]:
#matrix
x = np.array([[0, 3], [2, 2]])
x


array([[0, 3],
       [2, 2]])

In [12]:
x.shape

(2, 2)

The `.argsort()` returns the indices that would sort a matrix.  This can then be applied to sort an matrix, which is the same as `np.sort()`.  The purpose is to enable more complicated sorting routines.

In [17]:
ind = np.argsort(x, axis=0)    #sorts along first axis, rows (down), in ascending order
ind

array([[0, 1],
       [1, 0]])

In [18]:
np.take_along_axis(x, ind, axis=0)    #same as np.sort(x, axis=0)

array([[0, 2],
       [2, 3]])

### Accessing values

Indexing and accessing items is fairly intuitive, with value before each comma referring to an axis.  Notice with the `arr[0]`, below, that the earlier `np.sort(arr)` did not save inplace; therefore, it acts upon the original unsorted array.

We can slice a sequence of items, instead of an index, like this: [start:end], or with a pattern: [start:end:step].

In [19]:
arr[0]

2

In [20]:
x[1,1]

2

In [21]:
arr[0:3]

array([2, 1, 5])

In [22]:
arr[0:3:2]

array([2, 5])

In [23]:
x[0:1]    #the columns colon is implied
x[0:1,:]     

array([[0, 3]])

In [24]:
x[:,0:1]    #must use colon for rows, before comma

array([[0],
       [2]])

And we can reshape arrays and matrices, such as this example which flattens it.

In [25]:
x.reshape(-1)

array([0, 3, 2, 2])

### Boolean indexing

Instead of getting the values based on an index, we may want to perform the inverse of getting the index of values meeting a condition.

In [26]:
arr = np.array([1, 2, 3, 4, 5, 4, 4])
np.where(arr == 4)

(array([3, 5, 6]),)

Filters on an array are performed by using an array of the same dimension, but with boolean values.  

In [27]:
arr = np.array([41, 42, 43, 44])
x = [True, False, True, False]
arr[x]

array([41, 43])

The `.where()` method is combined with filter to create a 'mask' of which values fit selection criteria.

In [28]:
arr = np.array([1, 2, 3, 4, 5, 4, 4])
mask = arr > 3
arr[mask]

array([4, 5, 4, 4])

### Data types

Python’s floating-point numbers are usually 64-bit floating-point numbers.  Numpy improves the description of data types from simple `float` to a variety of different forms that effect memory allocation.  See the reference for [details](https://numpy.org/doc/stable/user/basics.types.html).

Python and Pandas use these data types:

* strings - used to represent text data, the text is given under quote marks. e.g. "ABCD"
* integer - used to represent integer numbers. e.g. -1, -2, -3
* float - used to represent real numbers. e.g. 1.2, 42.42
* boolean - used to represent True or False.
* complex - used to represent complex numbers. e.g. 1.0 + 2.0j, 1.5 + 2.5j

list of all data types in NumPy and the characters used to represent them.

* i - integer
* b - boolean
* u - unsigned integer
* f - float
* c - complex float
* m - timedelta
* M - datetime
* O - object
* S - string
* U - unicode string
* V - fixed chunk of memory for other type ( void )

TODO: add [information about arrow replacing numpy](https://wesmckinney.com/blog/apache-arrow-pandas-internals/)

Here, the `i` means signed integer. `4` represents a 4-byte size.

In [31]:
x = np.array([(1, 0), (0, 1)], dtype=[('x', '<i4'), ('y', '<i4')])
np.argsort(x, order=('x','y'))

array([(1, 0), (0, 1)], dtype=[('x', '<i4'), ('y', '<i4')])

In [30]:
np.argsort(x, order=('y','x'))

array([0, 1])

In [ ]:
# TODO: Vectorization

There is much more to numpy as it is foundational to scientific computing in python.  This is exemplified in its API being used by pytorch for working with tensors.

## Pandas Collections

The collections available through pandas are mostly convenience functions that make use of the optimized numpy functionality.  This greatly improves the speed of your workflow because many operations are so frequently reperformed.  When reviewing the pandas API, take note of the precedent that is set by numpy as pandas tries to follow this.

TODO: compare with [ddplyr](https://dplyr.tidyverse.org/)


### Creation

There are many ways to create pandas Series and DataFrames.  We will display loading data, dynamically, and through I/O operations.

In [7]:
#series from list
s = pd.Series([1, 3, 5, np.nan, 6, 8])
s

0    1.0
1    3.0
2    5.0
3    NaN
4    6.0
5    8.0
dtype: float64

In [8]:
#df from series
df = s.to_frame(name='mycol')
df

,mycol
0,1.0
1,3.0
2,5.0
3,NaN
4,6.0
5,8.0


Lists of records can be especially useful when integrating with data from other applications, such as JSON.

In [9]:
#list of dicts
trades = pd.DataFrame(
    [
        {"timestamp":"2016-05-25 13:30:01.023", "ticker":"MSFT", "price":51.95, "quantity":75},
        {"timestamp":"2016-05-25 13:30:01.038", "ticker":"MSFT", "price":51.95, "quantity":155},
        {"timestamp":"2016-05-25 13:30:03.048", "ticker":"GOOG", "price":720.77, "quantity":100},
        {"timestamp":"2016-05-25 13:30:03.048", "ticker":"GOOG", "price":720.92, "quantity":100},
        {"timestamp":"2016-05-25 13:30:03.048", "ticker":"AAPL", "price":98.00, "quantity":100},
    ]
)

In [44]:
#list of lists
trades = pd.DataFrame(
    [
        ["2016-05-25 13:30:01.023", "MSFT", 51.95, 75],
        ["2016-05-25 13:30:01.038", "MSFT", 51.95, 155],
        ["2016-05-25 13:30:03.048", "GOOG", 720.77, 100],
        ["2016-05-25 13:30:03.048", "GOOG", 720.92, 100],
        ["2016-05-25 13:30:03.048", "AAPL", 98.00, 100],
    ],
    columns=["timestamp", "ticker", "price", "quantity"],
)
trades['timestamp'] = pd.to_datetime(trades['timestamp'])
trades.head()

,timestamp,ticker,price,quantity
0,2016-05-25 13:30:01.023,MSFT,51.95,75
1,2016-05-25 13:30:01.038,MSFT,51.95,155
2,2016-05-25 13:30:03.048,GOOG,720.77,100
3,2016-05-25 13:30:03.048,GOOG,720.92,100
4,2016-05-25 13:30:03.048,AAPL,98.00,100


If reading from a file, then the `index_col` argument can be used to set the index as the data is being read.  This can improve performance as opposed to setting the index after it is initialized.

In [13]:
#csv
file_path = '../data/trades.csv'
trades_tmp = pd.read_csv(file_path, index_col='timestamp')
trades_tmp.shape

(5, 4)

### Selections

We will start by comparing against typical SQL queries.  The dataframe really shows its expressionful nature through brackets `[]`.  R and Pandas are similar in concept, but different in nuances.

* select columns: `SELECT column1, column2, ...FROM table_name;`
* select distinct: `SELECT DISTINCT column1, column2, ... FROM table_name;` 
* where (with AND, OR, NOT): `SELECT column1, column2, ... FROM table_name WHERE condition;`
* order by: `SELECT column1, column2, ... FROM table_name ORDER BY column1, column2, ... ASC|DESC;`
* insert into: `INSERT INTO table_name VALUES (value1, value2, value3, ...);`

Many of these methods come with the argument `inplace=False`, so you don't need to create a new dataframe at each step.

In [15]:
#select columns: `SELECT column1, column2, ...FROM table_name;`
trades[['ticker', 'price']]

,ticker,price
0,MSFT,51.95
1,MSFT,51.95
2,GOOG,720.77
3,GOOG,720.92
4,AAPL,98.00


In [16]:
#select distinct: `SELECT DISTINCT column1, column2, ... FROM table_name;`
trades[trades.duplicated(subset=['ticker'], keep='first')]

,timestamp,ticker,price,quantity
1,2016-05-25 13:30:01.038,MSFT,51.95,155
3,2016-05-25 13:30:03.048,GOOG,720.92,100


In [17]:
#where (with AND, OR, NOT): `SELECT column1, column2, ... FROM table_name WHERE condition;`
trades[ (trades['ticker']=='MSFT') & (trades['quantity']>75)]

,timestamp,ticker,price,quantity
1,2016-05-25 13:30:01.038,MSFT,51.95,155


In [18]:
trades[ (trades['ticker']=='MSFT') | (trades['quantity']<75)]

,timestamp,ticker,price,quantity
0,2016-05-25 13:30:01.023,MSFT,51.95,75
1,2016-05-25 13:30:01.038,MSFT,51.95,155


In [21]:
trades[ ~((trades['ticker']=='MSFT') & (trades['quantity']>75))]

,timestamp,ticker,price,quantity
0,2016-05-25 13:30:01.023,MSFT,51.95,75
2,2016-05-25 13:30:03.048,GOOG,720.77,100
3,2016-05-25 13:30:03.048,GOOG,720.92,100
4,2016-05-25 13:30:03.048,AAPL,98.00,100


In [22]:
#order by: `SELECT column1, column2, ... FROM table_name ORDER BY column1, column2, ... ASC|DESC;`
trades.sort_values(by=['ticker'], ascending=False, inplace=False)

,timestamp,ticker,price,quantity
0,2016-05-25 13:30:01.023,MSFT,51.95,75
1,2016-05-25 13:30:01.038,MSFT,51.95,155
2,2016-05-25 13:30:03.048,GOOG,720.77,100
3,2016-05-25 13:30:03.048,GOOG,720.92,100
4,2016-05-25 13:30:03.048,AAPL,98.00,100


Pandas is not quite as expressionful as R, here, as the `.loc()` method is needed to perform an insert.  However, the `.iloc()` allows rows to be selected by index, which R does not have available.

In [37]:
#insert into: `INSERT INTO table_name VALUES (value1, value2, value3, ...);`
tmp = trades.copy()
tmp.loc[tmp['quantity']==75, 'ticker'] = 'TEST'
tmp

,timestamp,ticker,price,quantity
0,2016-05-25 13:30:01.023,TEST,51.95,75
1,2016-05-25 13:30:01.038,MSFT,51.95,155
2,2016-05-25 13:30:03.048,GOOG,720.77,100
3,2016-05-25 13:30:03.048,GOOG,720.92,100
4,2016-05-25 13:30:03.048,AAPL,98.00,100


In [39]:
trades.iloc[1:3]
trades.iloc[[1,3]]

,timestamp,ticker,price,quantity
1,2016-05-25 13:30:01.038,MSFT,51.95,155
3,2016-05-25 13:30:03.048,GOOG,720.92,100


Note: subset replacement (the `SettingWithCopy` warning)

Conditionally replacing values in R is straightforward because we can use simultaneous selection.

In [40]:
%%R -i trades

tmp1 <- trades
tmp1[tmp1$quantity > 80, c('price')] <- 100
tmp1

            timestamp ticker  price quantity
0 2016-05-25 13:30:01   MSFT  51.95       75
1 2016-05-25 13:30:01   MSFT 100.00      155
2 2016-05-25 13:30:03   GOOG 100.00      100
3 2016-05-25 13:30:03   GOOG 100.00      100
4 2016-05-25 13:30:03   AAPL 100.00      100


In [41]:
tmp1 = trades.copy()
try:
    tmp1[tmp1.quantity > 80, 'price'] = 100
except:
    print("TypeError: 'Series' objects are mutable, thus they cannot be hashed")

TypeError: 'Series' objects are mutable, thus they cannot be hashed


The direct way fails, too!  Oh, pandas, why must you be so difficult...

In [42]:
tmp1 = trades.copy()
tmp1[tmp1.quantity > 80]['price'] = 100
tmp1

/tmp/ipykernel_1123/1674122135.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp1[tmp1.quantity > 80]['price'] = 100


,timestamp,ticker,price,quantity
0,2016-05-25 13:30:01.023,MSFT,51.95,75
1,2016-05-25 13:30:01.038,MSFT,51.95,155
2,2016-05-25 13:30:03.048,GOOG,720.77,100
3,2016-05-25 13:30:03.048,GOOG,720.92,100
4,2016-05-25 13:30:03.048,AAPL,98.00,100


Finally, the `.loc` method gives us what we want because we can use it with boolean indexing.

In [43]:
tmp1 = trades.copy()
tmp1.loc[tmp1.quantity > 80, 'price'] = 100
tmp1

,timestamp,ticker,price,quantity
0,2016-05-25 13:30:01.023,MSFT,51.95,75
1,2016-05-25 13:30:01.038,MSFT,100.00,155
2,2016-05-25 13:30:03.048,GOOG,100.00,100
3,2016-05-25 13:30:03.048,GOOG,100.00,100
4,2016-05-25 13:30:03.048,AAPL,100.00,100


### Row indexing

One of the fundamental differences between R and pandas is that pandas leans heavily upon the dataframe index.  For R users, it can seem like an annoyance; however, using it is required because many results of pandas methods return a dataframe with an index.



We can use integer indexing of columns, in R.  Its not necessary to use the `c()` concatenate function to create a vector, but it can often make your code cleaner.

In [46]:
%%R -i trades
trades[,c(2:3)]

  ticker  price
0   MSFT  51.95
1   MSFT  51.95
2   GOOG 720.77
3   GOOG 720.92
4   AAPL  98.00


However, the DataFrame indexing operator completely changes behavior to select rows when slice notation is used.  The DataFrame indexing operator selects rows in this manner, and can also do so by integer location, or by index label.

In [47]:
trades[:3]

,timestamp,ticker,price,quantity
0,2016-05-25 13:30:01.023,MSFT,51.95,75
1,2016-05-25 13:30:01.038,MSFT,51.95,155
2,2016-05-25 13:30:03.048,GOOG,720.77,100


The following selects rows beginning at integer location 1, up to but not including 6, by every third row.

In [48]:
trades[1:6:3]

,timestamp,ticker,price,quantity
1,2016-05-25 13:30:01.038,MSFT,51.95,155
4,2016-05-25 13:30:03.048,AAPL,98.00,100


You can also use slices consisting of string labels if your DataFrame index has strings in it.  Here we have new operators, `.iloc` to explicity support only integer indexing, and `.loc` to explicity support only label indexing.

In [49]:
trades.iloc[1:6:3]

,timestamp,ticker,price,quantity
1,2016-05-25 13:30:01.038,MSFT,51.95,155
4,2016-05-25 13:30:03.048,AAPL,98.00,100


The `.loc` / `.iloc` operators can select on both rows and columns, simultaneously, the indexing operator `[ ]` cannot.

In [50]:
trades.iloc[3:5, 0:2]

,timestamp,ticker
3,2016-05-25 13:30:03.048,GOOG
4,2016-05-25 13:30:03.048,AAPL


In [51]:
trades.iloc[[1, 2, 4], [0, 2]]

,timestamp,price
1,2016-05-25 13:30:01.038,51.95
2,2016-05-25 13:30:03.048,720.77
4,2016-05-25 13:30:03.048,98.00


With the timestamp index, we can subset using a variety of input string-formats.   When using a timestamp as an index, default datetime format ISO8601 (“yyyy-mm-dd hh:mm:ss”) is used when selecting data with partial string indexing. 

In [52]:
trades_idx.loc['2016-05-25 13:30:01':'2016-05-25 13:30:02']

,ticker,price,quantity
timestamp,,,
2016-05-25 13:30:01.023,MSFT,51.95,75
2016-05-25 13:30:01.038,MSFT,51.95,155


In [53]:
trades_idx.loc['2016-05']

,ticker,price,quantity
timestamp,,,
2016-05-25 13:30:01.023,MSFT,51.95,75
2016-05-25 13:30:01.038,MSFT,51.95,155
2016-05-25 13:30:03.048,GOOG,720.77,100
2016-05-25 13:30:03.048,GOOG,720.92,100
2016-05-25 13:30:03.048,AAPL,98.00,100


In [54]:
trades_idx.loc['May 2016']

,ticker,price,quantity
timestamp,,,
2016-05-25 13:30:01.023,MSFT,51.95,75
2016-05-25 13:30:01.038,MSFT,51.95,155
2016-05-25 13:30:03.048,GOOG,720.77,100
2016-05-25 13:30:03.048,GOOG,720.92,100
2016-05-25 13:30:03.048,AAPL,98.00,100


In [55]:
trades_idx.loc['May 2016', 'ticker']

timestamp
2016-05-25 13:30:01.023    MSFT
2016-05-25 13:30:01.038    MSFT
2016-05-25 13:30:03.048    GOOG
2016-05-25 13:30:03.048    GOOG
2016-05-25 13:30:03.048    AAPL
Name: ticker, dtype: object

The index is already set for `trades_idx`, but if we want to replace it with a new one, it is easily done.

In [56]:
trades_idx.set_index('price')

,ticker,quantity
price,,
51.95,MSFT,75
51.95,MSFT,155
720.77,GOOG,100
720.92,GOOG,100
98.00,AAPL,100


There are two ways to move the index into a column.

In [57]:
trades_idx.reset_index(level=0, inplace=False)

,timestamp,ticker,price,quantity
0,2016-05-25 13:30:01.023,MSFT,51.95,75
1,2016-05-25 13:30:01.038,MSFT,51.95,155
2,2016-05-25 13:30:03.048,GOOG,720.77,100
3,2016-05-25 13:30:03.048,GOOG,720.92,100
4,2016-05-25 13:30:03.048,AAPL,98.00,100


In [58]:
new_trades_idx = trades_idx.copy()
new_trades_idx['timestamp'] = trades_idx.index
new_trades_idx

,ticker,price,quantity,timestamp
timestamp,,,,
2016-05-25 13:30:01.023,MSFT,51.95,75,2016-05-25 13:30:01.023
2016-05-25 13:30:01.038,MSFT,51.95,155,2016-05-25 13:30:01.038
2016-05-25 13:30:03.048,GOOG,720.77,100,2016-05-25 13:30:03.048
2016-05-25 13:30:03.048,GOOG,720.92,100,2016-05-25 13:30:03.048
2016-05-25 13:30:03.048,AAPL,98.00,100,2016-05-25 13:30:03.048


While the index of a R dataframe is immutable, the index of a pandas dataframe can expand, or even lose arbitrary items.  In this example, some rows are removed, and the index goes with them.  If we don't `reset_index()`, then those rows will always be missing.

Note: forgetting to `reset_index()` after selections and filtering can cause serious problems, later.

In [59]:
tmp2 = trades.copy()
tmp3 = tmp2[0:4:2]
tmp3

,timestamp,ticker,price,quantity
0,2016-05-25 13:30:01.023,MSFT,51.95,75
2,2016-05-25 13:30:03.048,GOOG,720.77,100


In [60]:
tmp3.reset_index(inplace=True)
tmp3

,index,timestamp,ticker,price,quantity
0,0,2016-05-25 13:30:01.023,MSFT,51.95,75
1,2,2016-05-25 13:30:03.048,GOOG,720.77,100


You can also create a multi-index to make things even more ~~confusing~~ sophisticated.

In [61]:
index = pd.MultiIndex.from_product([['TX', 'FL', 'CA'], 
                                    ['North', 'South']], 
                                   names=['State', 'Direction'])

df = pd.DataFrame(index=index, 
                  data=np.random.randint(0, 10, (6,4)), 
                  columns=list('abcd'))
df

a  b  c  d
State Direction            
TX    North      4  7  1  3
      South      9  5  1  4
FL    North      2  3  5  9
      South      3  6  1  2
CA    North      0  7  8  2
      South      9  2  6  2

### Grouping operations

The `.groupby()` method is similar to that used in a SQL statement like the following:
```
SELECT Column1, Column2, mean(Column3), sum(Column4)
FROM SomeTable
GROUP BY Column1, Column2
```

However, it is much more flexible by breaking the process into three steps: split-apply-combine.  First, the dataframe is split into multiple dataframes using the provided criterion.  Group keys are sorted during this operation; however, `.groupby` will preserve the order in which observations are sorted within each group.  Also, `NA` values are excluded from group keys during the groupby operation.

Second, some function is applied to the groups.  Finally, the results are combined into a single dataframe. 

In [62]:
trades.groupby('ticker')['price'].sum()

ticker
AAPL      98.00
GOOG    1441.69
MSFT     103.90
Name: price, dtype: float64

In [63]:
gb = trades.groupby('ticker')
gb.groups

{'AAPL': [4], 'GOOG': [2, 3], 'MSFT': [0, 1]}

Now that groups are available, you can use the `.aggregate()` or (equivalent) `.agg()` methods to apply a function to each of the groups.

In [64]:
sm = gb.aggregate(np.sum)
sm

,price,quantity
ticker,,
AAPL,98.00,100
GOOG,1441.69,200
MSFT,103.90,230


To use the result as a dataframe without accompanying index, just `.reset_index()`.

In [65]:
sm.reset_index()

,ticker,price,quantity
0,AAPL,98.00,100
1,GOOG,1441.69,200
2,MSFT,103.90,230


Typical functions are apply are shown, below, but any user-provided function can be applied.

In [66]:
trades.groupby('ticker')['price'].agg(['count','sum','median','mean','std','max'])

,count,sum,median,mean,std,max
ticker,,,,,,
AAPL,1,98.00,98.000,98.000,NaN,98.00
GOOG,2,1441.69,720.845,720.845,0.106066,720.92
MSFT,2,103.90,51.950,51.950,0.000000,51.95


In [67]:
trades.groupby('ticker')['price'].agg('first','nth(1)')

ticker
AAPL     98.00
GOOG    720.77
MSFT     51.95
Name: price, dtype: float64

In [68]:
trades.groupby('ticker')['price'].agg(
    max = lambda x: x.max(),
    nth = lambda x: x.iat[0],                   #takes the first value
    #nth2 = lambda x: x.nth(0, dropna='any')    #why does this not work 
    ).reset_index()

,ticker,max,nth
0,AAPL,98.00,98.00
1,GOOG,720.92,720.77
2,MSFT,51.95,51.95


The transform method returns an object that is indexed the same as the one being grouped.  The transform must return a result that is either the same size as the group chunk or broadcastable to the size of the group chunk.

The steps below normalize and standardize the dataset.  We can verify this is done by performing another groupby with mean and standard deviation.

In [69]:
index = pd.date_range("10/1/1999", periods=1100)
ts = pd.Series(np.random.normal(0.5, 2, 1100), index)
ts = ts.rolling(window=100, min_periods=100).mean().dropna()

transformed = ts.groupby(lambda x: x.year).transform(
    lambda x: (x - x.mean()) / x.std()
)

In [70]:
transformed.groupby(lambda x: x.year).agg(['mean','std'])

,mean,std
2000,-4.745508e-16,1.0
2001,7.148011e-18,1.0
2002,-7.555128e-16,1.0


TODO: window and resample operations

### Merge

Merge with an indicator for where the rows came from

In [ ]:
df_merge = left.merge(right, on='key', how='left', indicator=True)

The `_merge` column is used to check for unexpected rows

In [5]:
df_merge._merge.value_counts()

NameError: name 'df_merge' is not defined

Merge by the nearest (not exact) timestamp

In [ ]:
pd.merge_asof(trades, quotes, on="timestamp", by='ticker', tolerance=pd.Timedelta('10ms'), direction='backward')

## General Pandas Usage

### Categorical data

TODO

### Timestamp data

A pandas object type is used for text or mixed numeric and non-numeric values. To get the correct order of the timestamp column we need to change it to the datetime64 type.

Replacing datetime values has its own unique manner, of course.  Don't want to lose this:

In [ ]:
df.timestamp = pd.to_datetime(df.timestamp.str.replace("D", "T"))

### Other data types

TODO

### Duplicates

TODO

### Missing values

Because Numpy has no native NA type, pandas uses multiple values denote a missing value:
* NaN for numeric/object missing values
* NaT for DateTime missing values
* None, which comes from Python

What surprised me is that None equals None in Python, but nan doesn’t equal nan in numpy.

In [ ]:
None == None

In [ ]:
pd.np.nan == pd.np.nan

This is important so that we don’t filter values by None:

In [ ]:
df[df.some_column == None]# instead use:
df[df.some_column.isnull()]

## Specific Examples

### Negate a string replacement

You can use negative look ahead `(?!)` assertion; `^(?!.*fish).*$` will firstly assert the pattern doesn't contain the word fish and then match every thing till the end of string and replace it with foo:

* `^` denotes the beginning of string, combined with `(?!.*fish)`, it asserts at BOS that there is no pattern like `.*fish` in the string;
* If the assertion succeeds, it matches everything till the end of string `.*$`, and replaces it with `foo`; If the assertion fails, the pattern doesn't match, nothing would happen;

so:

In [ ]:
df.replace(r'^(?!.*fish).*$', 'foo', regex=True)

### Remove Outliers

In [ ]:
df['size_clip'] = df['size'].clip(df['size'].quantile(0.01),
                                  df['size'].quantile(0.99))
df.size_clip.plot(kind='box')

### Bin / group data

In [ ]:
df['price_discrete_labels'] = pd.cut(df.price, 5, labels=['very low', 'low', 'mid', 'high', 'very high'])
df['price_discrete_labels'].value_counts()

Equal-sized groups

In [ ]:
df['price_discrete_equal_bins'] = pd.qcut(df.price, 5)
df['price_discrete_equal_bins'].value_counts()

## Conclusion

After reviewing these examples, we hope you agree that R is a terse, minimalistic, and powerful language.  You can do so many things with just the dataframe operator that it makes interacting with R a joy.  It is so much of a joy that I typically don't know how the operations are actually being performed - they just work.

Pandas is different.  It is bolted-on to Python, and its syntax is either not consistent or is just un-wieldly.  But maybe that is a good thing, because with Python I'm typically automating a process - not performing an interactive investigation.  I want to be forced to understand how the data is actually being worked, and I want to optimize operations for performance because it is a process that will be automated.

Let's stop the impassioned vitrioles of one clan against another, and use each of these tools for their respective strengths.